In [1]:
from scipy.stats import norm
import numpy as np

# Implement the closed form greeks for GBSM using python code. 

# Define the parameters for the option
current_stock_price = 151.03
strike_price = 165
risk_free_rate = 0.0425
continuously_compounding_coupon = 0.0053
current_date = np.datetime64('2022-03-13')
expiration_date = np.datetime64('2022-04-15')
time_to_expiration = (expiration_date - current_date).astype('timedelta64[D]').astype(int) / 365.0

# Black-Scholes formula components
def d1(S, K, T, r, q, sigma):
    return (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))

def d2(S, K, T, r, q, sigma):
    return d1(S, K, T, r, q, sigma) - sigma * np.sqrt(T)

# Greeks calculations
def black_scholes_greeks(S, K, T, r, q, sigma):
    D1 = d1(S, K, T, r, q, sigma)
    D2 = d2(S, K, T, r, q, sigma)
    call_delta = norm.cdf(D1)
    put_delta = -norm.cdf(-D1)
    gamma = norm.pdf(D1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(D1) * np.sqrt(T) * 0.01  # The vega is usually represented in percentage terms
    call_theta = (-S * sigma * norm.pdf(D1) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(D2)) / 365
    put_theta = (-S * sigma * norm.pdf(D1) / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-D2)) / 365
    call_rho = K * T * np.exp(-r * T) * norm.cdf(D2) * 0.01  # The rho is usually represented in percentage terms
    put_rho = -K * T * np.exp(-r * T) * norm.cdf(-D2) * 0.01

    return {
        'call_delta': call_delta,
        'put_delta': put_delta,
        'gamma': gamma,
        'vega': vega,
        'call_theta': call_theta,
        'put_theta': put_theta,
        'call_rho': call_rho,
        'put_rho': put_rho
    }

# Assume volatility is known, let's use an arbitrary volatility for demonstration purposes
volatility = 0.20  # Placeholder for volatility, this would usually be implied volatility

# Calculate the Greeks for both call and put options
greeks = black_scholes_greeks(current_stock_price, strike_price, time_to_expiration, risk_free_rate, continuously_compounding_coupon, volatility)

# Print the results
print("Greeks for Call:")
print(f"Delta: {greeks['call_delta']}")
print(f"Gamma: {greeks['gamma']}")
print(f"Vega: {greeks['vega']}")
print(f"Theta: {greeks['call_theta']}")
print(f"Rho: {greeks['call_rho']}")

print("\nGreeks for Put:")
print(f"Delta: {greeks['put_delta']}")
print(f"Gamma: {greeks['gamma']}")
print(f"Vega: {greeks['vega']}")
print(f"Theta: {greeks['put_theta']}")
print(f"Rho: {greeks['put_rho']}")


Greeks for Call:
Delta: 0.08301107089626869
Gamma: 0.016830979206204362
Vega: 0.06942036604441162
Theta: -0.022456481874505486
Rho: 0.011025939156368188

Greeks for Put:
Delta: -0.9169889291037313
Gamma: 0.016830979206204362
Vega: 0.06942036604441162
Theta: -0.0033178341735710703
Rho: -0.13758003122735787


In [2]:
# Implement a finite difference derivative calculation

# Define the parameters for the option
current_stock_price = 151.03
strike_price = 165
risk_free_rate = 0.0425
continuously_compounding_coupon = 0.0053
current_date = np.datetime64('2022-03-13')
expiration_date = np.datetime64('2022-04-15')
time_to_expiration = (expiration_date - current_date).astype('timedelta64[D]').astype(int) / 365

# Black-Scholes formula for European option price
def black_scholes_price(S, K, T, r, q, sigma, option_type='call'):
    D1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    D2 = D1 - sigma * np.sqrt(T)
    if option_type == 'call':
        price = S * np.exp(-q * T) * norm.cdf(D1) - K * np.exp(-r * T) * norm.cdf(D2)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-D2) - S * np.exp(-q * T) * norm.cdf(-D1)
    return price

# Define volatility, which would be estimated from market data (implied volatility)
volatility = 0.20  # Placeholder for volatility

# Finite difference approximation for the Greeks
def finite_difference_greeks(S, K, T, r, q, sigma, option_type='call'):
    h = 1e-5  # Small change for finite differences
    price = black_scholes_price(S, K, T, r, q, sigma, option_type)
    
    # Delta
    price_up = black_scholes_price(S + h, K, T, r, q, sigma, option_type)
    price_down = black_scholes_price(S - h, K, T, r, q, sigma, option_type)
    delta = (price_up - price_down) / (2 * h)
    
    # Gamma
    price_up_up = black_scholes_price(S + 2*h, K, T, r, q, sigma, option_type)
    price_down_down = black_scholes_price(S - 2*h, K, T, r, q, sigma, option_type)
    gamma = (price_up_up - 2*price + price_down_down) / (4 * h**2)
    
    # Theta
    T += 1/365  # Increase by one day
    price_T_up = black_scholes_price(S, K, T, r, q, sigma, option_type)
    theta = (price_T_up - price) / (1/365)
    
    # Vega (change in volatility)
    sigma += h
    price_sigma_up = black_scholes_price(S, K, T, r, q, sigma, option_type)
    vega = (price_sigma_up - price) / (h * 100)  # Presented in per 1% change in volatility
    
    # Rho (change in risk-free rate)
    r += h
    price_r_up = black_scholes_price(S, K, T, r, q, sigma, option_type)
    rho = (price_r_up - price) / (h * 100)  # Presented in per 1% change in risk-free rate
    
    return {
        'price': price,
        'delta': delta,
        'gamma': gamma,
        'theta': theta,
        'vega': vega,
        'rho': rho
    }

# Calculate finite difference Greeks for call and put options
fd_greeks_call = finite_difference_greeks(current_stock_price, strike_price, time_to_expiration, risk_free_rate, continuously_compounding_coupon, volatility, 'call')
fd_greeks_put = finite_difference_greeks(current_stock_price, strike_price, time_to_expiration, risk_free_rate, continuously_compounding_coupon, volatility, 'put')

# Print the results
print("Finite Difference Greeks for Call:")
print(f"Delta: {fd_greeks_call['delta']}")
print(f"Gamma: {fd_greeks_call['gamma']}")
print(f"Theta: {fd_greeks_call['theta']}")
print(f"Vega: {fd_greeks_call['vega']}")
print(f"Rho: {fd_greeks_call['rho']}")

print("\nFinite Difference Greeks for Put:")
print(f"Delta: {fd_greeks_put['delta']}")
print(f"Gamma: {fd_greeks_put['gamma']}")
print(f"Theta: {fd_greeks_put['theta']}")
print(f"Vega: {fd_greeks_put['vega']}")
print(f"Rho: {fd_greeks_put['rho']}")


Finite Difference Greeks for Call:
Delta: 0.08297130307255429
Gamma: 0.016817658377021868
Theta: 8.201791134286553
Vega: 22.543363068695754
Rho: 22.5552079749729

Finite Difference Greeks for Put:
Delta: -0.9165496337004696
Gamma: 0.01691091711109038
Theta: 2.016661124660999
Vega: 5.597801398522506
Rho: 5.456555021225995


In [3]:
# Implement the binomial tree valuation for American options with and without discrete dividends. Assume the stock above: Pays dividend on 4/11/2022 of $0.88

def binomial_tree_american(S, K, T, r, sigma, n, option_type, dividend_amount=0, dividend_dates=[]):
    # Time step
    dt = T / n
    # Calculate up and down factors
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    # Risk-neutral probability
    p = (np.exp((r - dividend_amount) * dt) - d) / (u - d)

    # Initialize arrays for stock prices and option values
    stock_prices = np.zeros((n+1, n+1))
    option_values = np.zeros((n+1, n+1))

    # Set up stock prices in the tree
    for i in range(n+1):
        for j in range(i+1):
            stock_prices[j, i] = S * (u ** (i-j)) * (d ** j)

    # Set up the last column with payoff at expiration
    for j in range(n+1):
        if option_type == 'call':
            option_values[j, n] = max(0, stock_prices[j, n] - K)
        else:  # put
            option_values[j, n] = max(0, K - stock_prices[j, n])

    # Calculate the option price at each node
    for i in range(n-1, -1, -1):
        for j in range(i+1):
            early_exercise = 0
            if option_type == 'call':
                early_exercise = max(0, stock_prices[j, i] - K)
            else:  # put
                early_exercise = max(0, K - stock_prices[j, i])

            # Adjust for dividends by reducing stock prices at ex-dividend dates
            if dividend_dates and i * dt < dividend_dates[0] <= (i+1) * dt:
                stock_prices[j, i] -= dividend_amount

            # Value of holding the option
            holding_value = (p * option_values[j, i+1] + (1 - p) * option_values[j+1, i+1]) * np.exp(-r * dt)

            # Option value is max of holding and exercising
            option_values[j, i] = max(early_exercise, holding_value)

    return option_values[0, 0]

# Parameters
S = 151.03  # Current stock price
K = 165  # Strike price
# Convert the time difference to float representing the number of days
current_date = np.datetime64('2022-03-13')
expiration_date = np.datetime64('2022-04-15')
T = (expiration_date - current_date).astype('timedelta64[D]').astype(int) / 365# Convert to fraction of year
r = 0.0425  # Risk-free rate
sigma = 0.80  # Volatility
n = 100  # Number of steps in the binomial tree
dividend_amount = 0.88  # Dividend amount
dividend_date = np.datetime64('2022-04-11').astype(int)  # Dividend payment date

# Calculate option values with dividend
call_value_with_dividend = binomial_tree_american(S, K, T, r, sigma, n, 'call', dividend_amount, [dividend_date])
put_value_with_dividend = binomial_tree_american(S, K, T, r, sigma, n, 'put', dividend_amount, [dividend_date])

# Calculate option values without dividend
call_value_no_dividend = binomial_tree_american(S, K, T, r, sigma, n, 'call', 0, [])
put_value_no_dividend = binomial_tree_american(S, K, T, r, sigma, n, 'put', 0, [])

# Print the results
print("Option Values with Dividend:")
print(f"Call Value: {call_value_with_dividend}")
print(f"Put Value: {put_value_with_dividend}")

print("\nOption Values without Dividend:")
print(f"Call Value: {call_value_no_dividend}")
print(f"Put Value: {put_value_no_dividend}")


Option Values with Dividend:
Call Value: 5.943325900144015
Put Value: 30.226821632852445

Option Values without Dividend:
Call Value: 9.349599724132323
Put Value: 22.756696757667296


In [4]:
# Calculate the Greeks of each

def binomial_tree_greeks(S, K, T, r, sigma, n, option_type, dividend_amount, dividend_dates):
    # Calculate the base option price
    base_price = binomial_tree_american(S, K, T, r, sigma, n, option_type, dividend_amount, dividend_dates)
    
    # Delta and Gamma
    dS = S * 0.01
    price_up = binomial_tree_american(S + dS, K, T, r, sigma, n, option_type, dividend_amount, dividend_dates)
    price_down = binomial_tree_american(S - dS, K, T, r, sigma, n, option_type, dividend_amount, dividend_dates)
    delta = (price_up - price_down) / (2 * dS)
    gamma = (price_up - 2 * base_price + price_down) / (dS ** 2)
    
    # Theta
    dt = 1/365
    price_tomorrow = binomial_tree_american(S, K, T - dt, r, sigma, n, option_type, dividend_amount, dividend_dates)
    theta = (price_tomorrow - base_price) / dt
    
    # Vega
    dvol = sigma * 0.01
    price_vol_up = binomial_tree_american(S, K, T, r, sigma + dvol, n, option_type, dividend_amount, dividend_dates)
    vega = (price_vol_up - base_price) / dvol
    
    # Rho
    dr = 0.0001
    price_r_up = binomial_tree_american(S, K, T, r + dr, sigma, n, option_type, dividend_amount, dividend_dates)
    rho = (price_r_up - base_price) / dr
    
    return {
        'price': base_price,
        'delta': delta,
        'gamma': gamma,
        'theta': theta,
        'vega': vega,
        'rho': rho
    }

# Calculate Greeks for call and put with and without dividends
greeks_call_with_dividend = binomial_tree_greeks(S, K, T, r, sigma, n, 'call', dividend_amount, [dividend_date])
greeks_put_with_dividend = binomial_tree_greeks(S, K, T, r, sigma, n, 'put', dividend_amount, [dividend_date])

greeks_call_no_dividend = binomial_tree_greeks(S, K, T, r, sigma, n, 'call', 0, [])
greeks_put_no_dividend = binomial_tree_greeks(S, K, T, r, sigma, n, 'put', 0, [])

# Print the results
print("Greeks for Call with Dividend:")
print(f"Delta: {greeks_call_with_dividend['delta']}")
print(f"Gamma: {greeks_call_with_dividend['gamma']}")
print(f"Theta: {greeks_call_with_dividend['theta']}")
print(f"Vega: {greeks_call_with_dividend['vega']}")
print(f"Rho: {greeks_call_with_dividend['rho']}")

print("\nGreeks for Put with Dividend:")
print(f"Delta: {greeks_put_with_dividend['delta']}")
print(f"Gamma: {greeks_put_with_dividend['gamma']}")
print(f"Theta: {greeks_put_with_dividend['theta']}")
print(f"Vega: {greeks_put_with_dividend['vega']}")
print(f"Rho: {greeks_put_with_dividend['rho']}")

print("\nGreeks for Call without Dividend:")
print(f"Delta: {greeks_call_no_dividend['delta']}")
print(f"Gamma: {greeks_call_no_dividend['gamma']}")
print(f"Theta: {greeks_call_no_dividend['theta']}")
print(f"Vega: {greeks_call_no_dividend['vega']}")
print(f"Rho: {greeks_call_no_dividend['rho']}")

print("\nGreeks for Put without Dividend:")
print(f"Delta: {greeks_put_no_dividend['delta']}")
print(f"Gamma: {greeks_put_no_dividend['gamma']}")
print(f"Theta: {greeks_put_no_dividend['theta']}")
print(f"Vega: {greeks_put_no_dividend['vega']}")
print(f"Rho: {greeks_put_no_dividend['rho']}")



Greeks for Call with Dividend:
Delta: 0.31491723133259686
Gamma: 0.008035264186402614
Theta: -40.664020637951275
Vega: 15.300335670813347
Rho: 2.6123541553424445

Greeks for Put with Dividend:
Delta: -0.6439305735493484
Gamma: 0.009297238052048256
Theta: -147.55478851498364
Vega: 14.673856245393413
Rho: -11.72046488687073

Greeks for Call without Dividend:
Delta: 0.42641967654359736
Gamma: 0.011603243083945261
Theta: -82.86069401391454
Vega: 18.08344348373936
Rho: 4.728038087513653

Greeks for Put without Dividend:
Delta: -0.577471616786322
Gamma: 0.013017980071740421
Theta: -76.51746265375296
Vega: 18.024328802098033
Rho: -8.188856681350387
